In [23]:
import sqlite3
import pandas as pd 
import requests

con =sqlite3.connect('database.sqlite')
cursor = con.cursor()

cursor.execute(
"""
SELECT name
FROM sqlite_master
WHERE type='table';
"""
)
print(cursor.fetchall())

[('sqlite_sequence',), ('Matches',), ('Teams_in_Matches',), ('Teams',), ('Unique_Teams',)]


In [36]:
df_new = pd.read_sql_query(f"""


    WITH STEP1 AS (SELECT  DISTINCT m.Match_ID, m.Season, 
    m.Date, m.HomeTeam as TeamName,  m.FTHG AS Goal, m.Div,
    CASE WHEN m.FTHG > FTAG THEN 1
                     
                     ELSE 0
                     END AS Win
    FROM Matches AS m
    WHERE m.season = '2009'
    
    UNION 
    SELECT  DISTINCT m.Match_ID, m.Season, 
    m.Date, m.AwayTeam as TeamName, m.FTAG AS Goal, m.Div,
       CASE WHEN m.FTHG < FTAG THEN 1
                     
                     ELSE 0
                     END AS Win_AwayTeam
  
    FROM Matches AS m
    WHERE m.season = '2009'
    ORDER BY m.Match_ID)
    
    SELECT DISTINCT TeamName, Div, Season, Date, SUM(Goal) OVER (PARTITION BY TeamName) as Total_Goal, 
    SUM(Win) OVER (PARTITION BY TeamName) as Total_Win,
    COUNT() OVER (PARTITION BY TeamName) as Total_Game
    
    FROM STEP1
    """, con)
df_new

,TeamName,Div,Season,Date,Total_Goal,Total_Win,Total_Game
0,Aachen,D2,2009,2010-01-15,37,11,34
1,Aachen,D2,2009,2009-11-09,37,11,34
2,Aachen,D2,2009,2009-09-20,37,11,34
3,Aachen,D2,2009,2009-08-17,37,11,34
4,Aachen,D2,2009,2010-03-12,37,11,34
...,...,...,...,...,...,...,...
1979,Wolves,E0,2009,2010-04-11,32,9,38
1980,Wolves,E0,2009,2010-04-17,32,9,38
1981,Wolves,E0,2009,2010-04-24,32,9,38
1982,Wolves,E0,2009,2010-05-01,32,9,38


In [17]:
def pipeline(season):
    df_new = pd.read_sql_query(f"""SELECT  DISTINCT m.Match_ID, m.Season, 
    m.Date, m.HomeTeam, m.AwayTeam, m.FTHG, m.FTAG, m.Div,
    CASE WHEN m.FTHG > FTAG THEN m.HomeTeam
                     WHEN m.FTHG < FTAG THEN m.AwayTeam
                     ELSE 'TIE'
                     END AS Win_Team,
     CASE WHEN m.FTHG < FTAG THEN m.HomeTeam
                     WHEN m.FTHG > FTAG THEN m.AwayTeam
                     ELSE 'TIE'
                     END AS Lose_Team
    FROM Matches AS m
    JOIN Teams_IN_matches AS tm ON M.Match_ID = tm.Match_ID
    JOIN Unique_Teams AS ut ON tm.Unique_Team_ID = ut.Unique_Team_ID
    JOIN Teams as t on ut.TeamName = t.TeamName
    WHERE m.season = {season}
    ORDER BY m.Match_ID 
    LIMIT 5
    """, con)
    df_new['New_Date'] = df_new['Date']+'T00:00:00'
    
    urls=[]   
    for date in df_new['New_Date']:
        urls.append(f'https://api.darksky.net/forecast/{password}/52.52,13.405,{date}?exclude=minutely,hourly, alerts, flags')   
        pt_dict =[]
        for url in urls:
            response = requests.get(url)
            if response.status_code == 200:
                response_dict = response.json()
                pt_dict.append(response_dict['daily']['data'][0]['precipIntensity'])
    df_new['Rain']= pt_dict
    df_new['Rain'][df_new['Rain'] >0]='Not Rain'
    df_new['Rain'][df_new['Rain'] == 0]='Rain'
    return df_new

df_new = pipeline(2009)
df_new

/Users/daihongchen/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/daihongchen/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Match_ID,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,Div,Win_Team,Lose_Team,New_Date,Rain
0,1,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,D2,Oberhausen,Kaiserslautern,2010-04-04T00:00:00,Rain
1,2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,D2,Kaiserslautern,Munich 1860,2009-11-01T00:00:00,Rain
2,3,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D2,TIE,TIE,2009-10-04T00:00:00,Not Rain
3,4,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,D2,Frankfurt FSV,Karlsruhe,2010-02-21T00:00:00,Rain
4,5,2009,2009-12-06,Ahlen,Karlsruhe,1,3,D2,Karlsruhe,Ahlen,2009-12-06T00:00:00,Not Rain
